In [ ]:
from ipywidgets import interact, FloatSlider, widgets, fixed
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,AutoMinorLocator
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
from bleacher import simulation
import math
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 12 

## Determination of pattern of light sources

In [ ]:
# Define sliders for interactive controls using ipywidgets
width_slider = FloatSlider(min=10, max=500, step=5, value=300, description='Width')
height_slider = FloatSlider(min=10, max=500, step=5, value=200, description='Height')
dx_slider = FloatSlider(min=1, max=50, step=1, value=15, description='dx')
dy_slider = FloatSlider(min=1, max=50, step=1, value=15, description='dy')

# Create an interactive plot using interact
interact(simulation.plot_grid, width=width_slider, height=height_slider, dx=dx_slider, dy=dy_slider);

## Analysis of the interval effect

In [ ]:
# set parameters
width = 300
height = 200
# dx = 25 # set the range at the next
dy_fixed = 25
depth = 20
phi = 100 # lumen
unit = 0.001 # mm
illumination = 'lambertian' # either lambertian or point

In [ ]:
# prepare plotting
fig = plt.figure(figsize=(8, 8))
col = 2
row = 2
gs = GridSpec(row, col, width_ratios=[6, 6])

ax_list = [[],[]]
# left-top plot for the fixed dy
# right-top plot for the prediction of fixed dy
# left-bottom plot for the both moving dx and dy 
# right-bottom plot for the prediction of both moving dx and dy 
for r in range(row):
    for c in range(col):
        ax_list[r].append(fig.add_subplot(gs[r,c]))

# set dx_range
dx_range = range(3,50,1)

# set depth range
depth_range = range(10,100,25)

# Create a colormap object
cmap = plt.get_cmap('cool', len(list(depth_range)))

# from scipy.optimize import curve_fit # if fitting is required

for i, depth in enumerate(depth_range):
    # set observation point
    obs_point = np.asarray([0,0,depth])
    x_vals = np.asarray(list(dx_range))
    
    result_dx = []
    # for fixed dy
    for dx in dx_range:
        dy = dy_fixed
        points = simulation.generate_points(width, height, dx, dy)
        result_dx.append(simulation.compute_sum_at_obs_point(points, obs_point, phi, unit, illumination=illumination))
    # for dx = dy
    result_dxdy = []
    for dx in dx_range:
        dy = dx
        points = simulation.generate_points(width, height, dx, dy)
        result_dxdy.append(simulation.compute_sum_at_obs_point(points, obs_point, phi, unit, illumination=illumination))

    """
    # set function to fit
    def inverse_function(x, a):
        return a / x
    
    def inverse_square_function(x, a):
        return a / (x**2)
    
    # Fit the data
    popt, pcov = curve_fit(inverse_function, x_vals, result_dx)
    # Prediction
    y_pred = inverse_function(x_vals, popt)
    # Fit the data
    popt, pcov = curve_fit(inverse_square_function, x_vals, result_dxdy)
    # Prediction
    y_pred = inverse_square_function(x_vals, popt)
    """

    color = cmap(i)
    # plot
    ax_list[0][0].plot(x_vals,result_dx, color=color)
    ax_list[0][1].plot(1/x_vals,result_dx, color=color)
    
    # plot
    ax_list[1][0].plot(x_vals,result_dxdy, color=color)
    ax_list[1][1].plot(1/(x_vals**2),result_dxdy, color=color)

for r in range(row):
    for c in range(col):
        ax_list[r][c].set_ylim(0,)
        ax_list[r][c].set_xlim(0,)
        ax_list[r][c].set_xlabel('Interval')
        ax_list[r][c].set_ylabel('Total illuminance')
        ax_list[r][c].ticklabel_format(axis='y',style='scientific', scilimits=(0,0))
        ax_list[r][c].legend(list(depth_range),loc="upper right")
        ax_list[r][c].spines[['right', 'top']].set_visible(False)

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/intervals.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as svg")
else:
    print("No figure to export.")

### D curve and intervals

In [ ]:
# set parameters
width = 300
height = 200
# dx = 25 # set the range at the next
dy_fixed = 25
# depth = 20 set the range at the next
phi = 100 # lumen
unit = 0.001 # mm
illumination = 'lambertian' # either lambertian or point

In [ ]:
# prepare plotting
fig, ax = plt.subplots(figsize=(3, 3))

# set dx_range
dx_range = range(15,40,5)

# set depth range
depth_range = list(range(10,100,25))

# Create a colormap object
cmap = plt.get_cmap('summer', len(list(dx_range)))

for i, dx in enumerate(dx_range):
    dy = dx

    points = simulation.generate_points(width, height, dx, dy)

    result_depth = []
    for depth in depth_range:
        # set observation point
        obs_point = np.asarray([0,0,depth])
        result_depth.append(simulation.compute_sum_at_obs_point(points, obs_point, phi, unit, illumination=illumination))

    color = cmap(i)
    # plot
    ax.plot(np.asarray(list(depth_range)),result_depth, color=color)


ax.set_ylim(0,)
ax.set_xlim(0,)
ax.set_xlabel('Depth')
ax.set_ylabel('Total illuminance')
ax.ticklabel_format(axis='y',style='scientific', scilimits=(0,0))
ax.legend(list(dx_range),loc="upper right")
ax.spines[['right', 'top']].set_visible(False)

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/depth_intervals.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as svg")
else:
    print("No figure to export.")

## Analysis of the board size and interval effects on z-profile

### Board size effect

In [ ]:
# set parameters
# width = 300 
# height = width # set the range at the next
dx = 15
dy = 15
# depth = 20
phi = 100 # lumen
unit = 0.001 # mm
illumination = 'lambertian' # either lambertian or point

In [ ]:
# prepare plotting
fig = plt.figure(figsize=(7, 3))
col = 2
row = 1
gs = GridSpec(row, col, width_ratios=[6, 6])

ax_list = [[],[]]

for r in range(row):
    for c in range(col):
        ax_list[r].append(fig.add_subplot(gs[r,c]))

# set width_range
width_range = [100,300,1000,2500,10000]# list(range(100,1700,200))

# set depth range
depth_range = range(10,90,1)

# Create a colormap object
cmap = plt.get_cmap('winter', len(width_range))

for i, width in enumerate(width_range):
    height = width
    results = []
    
    # for fixed dy
    for depth in depth_range:
        # set observation point
        obs_point = np.asarray([0,0,depth])
        points = simulation.generate_points(width, height, dx, dy)
        results.append(simulation.compute_sum_at_obs_point(points, obs_point, phi, unit, illumination=illumination))

    results_both_side = [results[i] + results[-i-1] for i in range(len(results))]
    
    color = cmap(i)
    # plot
    ax_list[0][0].plot(list(depth_range),results, color=color)
    ax_list[0][1].plot(list(depth_range),results_both_side, color=color)

for r in range(row):
    for c in range(col):
        ax_list[r][c].set_ylim(0,)
        ax_list[r][c].set_xlim(0,100)
        ax_list[r][c].set_xlabel('Depth')
        ax_list[r][c].set_ylabel('Total illuminance')
        ax_list[r][c].ticklabel_format(axis='y',style='scientific', scilimits=(0,0))
        ax_list[r][c].legend(width_range,loc="upper right")
        ax_list[r][c].spines[['right', 'top']].set_visible(False)


In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/width_height_15intervals.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as svg")
else:
    print("No figure to export.")

## Hot spot analysis

### Precompute the 3D heatmap array

In [ ]:
# set parameters
width = 300 # v2: 300, Tsuneoka: 260, 
height = 200 # v2: 200, Tsuneoka: 100, 
dx = 15 # v2: 12.5, Tsuneoka: 40, super: 14
dy = 15 # v2: 12.5, Tsuneoka: 40, super: 14
phi = 100 # lumen of the LED. v2: 223 lm, Tsuneoka: 4131 lm, Super: 3791?
unit = 0.001 # 0.001 for mm, 0.01 for cm, 1.0 for m

board_position = 0.0
illumination = 'lambertian' # either lambertian or point
grid_constant = 1.0 # sampling frequency in xy

In [ ]:
# Precompute heatmaps for a range of depth values
depth_range = np.linspace(1, 100, 100)  # Define the range of depth values to precompute
precomputed_heatmaps, _, _ = simulation.precompute_heatmaps(
    width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
    board_position=board_position, grid_constant=grid_constant, illumination=illumination
)

# Fix nan or inf
precomputed_heatmaps = np.nan_to_num(precomputed_heatmaps)

### Interactive analysis of hot spot

In [ ]:
# Function to update the plots based on selected centered y and z positions
def update_plot_yz(y_centered, z_position, grid_constant=1.0, contour=False, contour_percent=85):
    global last_fig  # To store the figure globally
    # Convert centered y coordinate back to array index
    y_position = y_centered // int(grid_constant) + precomputed_heatmaps.shape[1] // 2

    # Create a figure with GridSpec
    fig = plt.figure(figsize=(12, 6))
    gs = GridSpec(1, 3, width_ratios=[6, 0.3, 6]) 

    # First plot: XY plane at the selected z position with red line at the centered y position
    ax0 = fig.add_subplot(gs[0])
    heatmap = ax0.imshow(precomputed_heatmaps[z_position], cmap='viridis', 
               extent=[
                   -precomputed_heatmaps.shape[2]*grid_constant//2, 
                   precomputed_heatmaps.shape[2]*grid_constant//2, 
                   -precomputed_heatmaps.shape[1]*grid_constant//2, 
                   precomputed_heatmaps.shape[1]*grid_constant//2
               ],
               vmin=0,vmax=np.percentile(precomputed_heatmaps,95))

    # Overlay the contour at 90% of the max value
    # Calculate the 90% of the max value for the contour level
    if contour:
        contour_level = contour_percent/100. * np.max(precomputed_heatmaps[z_position])
        ax0.contour(precomputed_heatmaps[z_position], 
                    levels=[contour_level], colors='gray', linewidths=1.5, linestyles='dashed', 
                    extent=[
                        -precomputed_heatmaps.shape[2]*grid_constant//2, 
                        precomputed_heatmaps.shape[2]*grid_constant//2, 
                        -precomputed_heatmaps.shape[1]*grid_constant//2, 
                        precomputed_heatmaps.shape[1]*grid_constant//2
                    ])

    ax0.axhline(y=y_centered, color='red', linewidth=2)  # Red line indicating centered y position
    ax0.set_title(f'XY plane at z={z_position} with y={y_centered} (centered) marked')
    ax0.set_xlabel('X (centered)')
    ax0.set_ylabel('Y (centered)')
    ax0.set_aspect('equal')
    
    # Add colorbar to the right of the heatmap
    cbar_ax = fig.add_subplot(gs[1])  # Add the colorbar axis
    plt.colorbar(heatmap, cax=cbar_ax)

    # Second plot: X profile at the selected centered y and z position
    ax1 = fig.add_subplot(gs[2])
    ax1.plot(precomputed_heatmaps[z_position, y_position], label=f'Profile at (y={y_centered}, z={z_position})')
    ax1.set_ylim([0, np.percentile(precomputed_heatmaps,99)])
    ax1.set_xlabel('X')
    ax1.set_ylabel('Value')
    ax1.set_title('X-direction profile')
    ax1.ticklabel_format(axis='y',style='scientific', scilimits=(0,0))
    ax1.legend()

    plt.tight_layout()
    plt.show()

    last_fig = fig

In [ ]:
# Create sliders for z position and centered y position
z_slider = widgets.IntSlider(
    min=0, max=precomputed_heatmaps.shape[0] - 1, 
    step=1, description='Z position'
)
y_slider = widgets.IntSlider(
    min=-precomputed_heatmaps.shape[1]*grid_constant//2, max=precomputed_heatmaps.shape[1]*grid_constant//2 - math.ceil(grid_constant), 
    step=1, description='Y position (centered)'
)
contour_slider = widgets.IntSlider(
    min=0, max=100, value=75, 
    step=1, description='contour %'
)
# Interactive plot update
interact(update_plot_yz, y_centered=y_slider, z_position=z_slider, grid_constant=fixed(grid_constant), contour_percent=contour_slider)

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/heatmap_dx15_z20_contour.svg'

# Save to an SVG file
if last_fig:
    last_fig.savefig(savepath, format='svg')
    print("Plot saved as svg")
else:
    print("No figure to export.")

### Dual side illumination analysis of heatmap

In [ ]:
board_position_02 = 100
precomputed_heatmaps_02, _, _ = simulation.precompute_heatmaps(
    width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
    board_position=board_position_02, grid_constant=grid_constant, illumination=illumination
)

# Fix nan or inf
precomputed_heatmaps_02 = np.nan_to_num(precomputed_heatmaps_02)

# combine single side and dual side
precomputed_heatmaps = precomputed_heatmaps + precomputed_heatmaps_02

In [ ]:
# Create sliders for z position and centered y position
z_slider = widgets.IntSlider(
    min=0, max=precomputed_heatmaps.shape[0] - 1, 
    step=1, description='Z position'
)
y_slider = widgets.IntSlider(
    min=-precomputed_heatmaps.shape[1]*grid_constant//2, max=precomputed_heatmaps.shape[1]*grid_constant//2 - math.ceil(grid_constant), 
    step=1, description='Y position (centered)'
)
contour_slider = widgets.IntSlider(
    min=0, max=100, value=75, 
    step=1, description='contour %'
)
# Interactive plot update
interact(update_plot_yz, y_centered=y_slider, z_position=z_slider, grid_constant=fixed(grid_constant), contour_percent=contour_slider)

### Variance-to-mean ratio plotting

In [ ]:
# set parameters for the plotting
y_centered = 0
x_range = [125,175]
depth_range = np.linspace(1, 100, 100)
dx_range = [10,15,25,35]

In [ ]:
# plot single-sided simulation
fig, ax = plt.subplots(figsize=(3, 3))


# Create a colormap object
cmap = plt.get_cmap('cool', len(dx_range))

for i, dx in enumerate(dx_range):
    dy = dx
    precomputed_heatmaps, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position, grid_constant=grid_constant, illumination=illumination
    )

    # Fix nan or inf
    precomputed_heatmaps = np.nan_to_num(precomputed_heatmaps)
    
    y_position = y_centered // int(grid_constant) + precomputed_heatmaps.shape[1] // 2
    variances = np.var(precomputed_heatmaps[:,y_position,x_range[0]:x_range[1]], axis=1)
    means = np.mean(precomputed_heatmaps[:,y_position,x_range[0]:x_range[1]], axis=1)
    v2m = variances / means
    color = cmap(i)
    ax.plot(v2m, color=color)

    
ax.set_xlim(0,)
ax.set_ylim(0,100)
ax.set_xlabel('Depth')
ax.set_ylabel('variance-to-mean ratio')
ax.xaxis.set_major_locator(MultipleLocator(50))
ax.xaxis.set_minor_locator(AutoMinorLocator()) 
ax.yaxis.set_major_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(AutoMinorLocator()) 
ax.set_title(f'v2m at y={y_centered}')
ax.legend(dx_range,loc="upper right")

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/v2m.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as 'exported_plot.svg'")
else:
    print("No figure to export.")

In [ ]:
# plot dual-sided simulation
# set parameters
board_position = 0.0
board_position_02 = 100.

fig, ax = plt.subplots(figsize=(3, 3))

# Create a colormap object
cmap = plt.get_cmap('cool', len(dx_range))

for i, dx in enumerate(dx_range):
    dy = dx
    precomputed_heatmaps1, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position, grid_constant=grid_constant, illumination=illumination
    )
    precomputed_heatmaps2, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position_02, grid_constant=grid_constant, illumination=illumination
    )

    precomputed_heatmaps = precomputed_heatmaps1 + precomputed_heatmaps2

    # Fix nan or inf
    precomputed_heatmaps = np.nan_to_num(precomputed_heatmaps)
    
    y_position = y_centered // int(grid_constant) + precomputed_heatmaps.shape[1] // 2
    variances = np.var(precomputed_heatmaps[:,y_position,x_range[0]:x_range[1]], axis=1)
    means = np.mean(precomputed_heatmaps[:,y_position,x_range[0]:x_range[1]], axis=1)
    v2m = variances / means
    color = cmap(i)
    ax.plot(v2m, color=color)

    
ax.set_xlim(0,)
ax.set_ylim(0,100)
ax.set_xlabel('Depth')
ax.set_ylabel('variance-to-mean ratio')
ax.xaxis.set_major_locator(MultipleLocator(50))
ax.xaxis.set_minor_locator(AutoMinorLocator()) 
ax.yaxis.set_major_locator(MultipleLocator(50))
ax.yaxis.set_minor_locator(AutoMinorLocator()) 
ax.set_title(f'v2m at y={y_centered}')
ax.legend(dx_range,loc="upper right")

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/v2m_dual.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as 'exported_plot.svg'")
else:
    print("No figure to export.")

### Illumination coverage plotting

In [ ]:
# set parameters for the plotting
percentage = 75
z_reference_position=40

fig, ax = plt.subplots(figsize=(3, 3))

# Create a colormap object
cmap = plt.get_cmap('cool', len(dx_range))

for j, dx in enumerate(dx_range):
    dy = dx
    precomputed_heatmaps, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position, grid_constant=grid_constant, illumination=illumination
    )

    # Fix nan or inf
    precomputed_heatmaps = np.nan_to_num(precomputed_heatmaps)
    
    coverages = []

    reference = np.max(precomputed_heatmaps[z_reference_position,:,:])
    for i in range(precomputed_heatmaps.shape[0]):
        plane = precomputed_heatmaps[i,:,:]
        threshold = percentage / 100. * reference# np.max(plane)
        covered_area = (plane>threshold).sum()
        coverage = covered_area / plane.size
        coverages.append(coverage)

    color = cmap(j)
    ax.plot(coverages, color=color)


ax.set_xlim(0,)
ax.set_ylim(0,1.0)
ax.xaxis.set_major_locator(MultipleLocator(50)) # Intervals for major x-ticks                                                                                                                                                                                                                     
ax.xaxis.set_minor_locator(AutoMinorLocator()) 
ax.yaxis.set_major_locator(MultipleLocator(0.5)) # Intervals for major x-ticks                                                                                                                                                                                                                     
ax.yaxis.set_minor_locator(AutoMinorLocator()) 
ax.set_xlabel('Depth')
ax.set_ylabel('Coverage')
ax.set_title(f'Coverage at percent more than {percentage}%')
ax.legend(dx_range,loc="upper right")

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/coverage_reference_40.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as 'exported_plot.svg'")
else:
    print("No figure to export.")

In [ ]:
# set parameters for the plotting
percentage = 75
z_reference_position=40
board_position = 0.0
board_position_02 = 100.

fig, ax = plt.subplots(figsize=(3, 3))

# Create a colormap object
cmap = plt.get_cmap('cool', len(dx_range))

for j, dx in enumerate(dx_range):
    dy = dx
    precomputed_heatmaps1, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position, grid_constant=grid_constant, illumination=illumination
    )
    precomputed_heatmaps2, _, _ = simulation.precompute_heatmaps(
        width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
        board_position=board_position_02, grid_constant=grid_constant, illumination=illumination
    )

    precomputed_heatmaps = precomputed_heatmaps1 + precomputed_heatmaps2
    
    coverages = []
    reference = np.max(precomputed_heatmaps[z_reference_position,:,:])
    for i in range(precomputed_heatmaps.shape[0]):
        plane = precomputed_heatmaps[i,:,:]
        threshold = percentage / 100. * reference# np.max(plane)
        covered_area = (plane>threshold).sum()
        coverage = covered_area / plane.size
        coverages.append(coverage)

    color = cmap(j)
    ax.plot(coverages, color=color)


ax.set_xlim(0,)
ax.set_ylim(0,1.0)
ax.xaxis.set_major_locator(MultipleLocator(50)) # Intervals for major x-ticks                                                                                                                                                                                                                     
ax.xaxis.set_minor_locator(AutoMinorLocator()) 
ax.yaxis.set_major_locator(MultipleLocator(0.5)) # Intervals for major x-ticks                                                                                                                                                                                                                     
ax.yaxis.set_minor_locator(AutoMinorLocator()) 
ax.set_xlabel('Depth')
ax.set_ylabel('Coverage')
ax.set_title(f'Coverage at percent more than {percentage}%')
ax.legend(dx_range,loc="upper right")

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/coverage_dual_reference_40.svg'

# Save to an SVG file
if fig:
    fig.savefig(savepath, format='svg')
    print("Plot saved as 'exported_plot.svg'")
else:
    print("No figure to export.")

## Simulation for the final configuration

In [ ]:
# set parameters
width = 300 # v2: 300, Tsuneoka: 260, 
height = 200 # v2: 200, Tsuneoka: 100, 
dx = 12.5 # v2: 12.5, Tsuneoka: 40, super: 14
dy = 12.5 # v2: 12.5, Tsuneoka: 40, super: 14
phi = 223 # lumen of the LED. v2: 223 lm, Tsuneoka: 4131 lm, Super: 3791?
unit = 0.001 # 0.001 for mm, 0.01 for cm, 1.0 for m
depth_range = np.linspace(1, 110, 110)

board_position = 0.0
board_position_02 = 110

illumination = 'lambertian' # either lambertian or point
grid_constant = 1.0 # sampling frequency in xy

precomputed_heatmaps, _, _ = simulation.precompute_heatmaps(
    width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
    board_position=board_position, grid_constant=grid_constant, illumination=illumination
)

precomputed_heatmaps_02, _, _ = simulation.precompute_heatmaps(
    width=width, height=height, dx=dx, dy=dy, phi=phi, depth_range=depth_range, unit=unit,
    board_position=board_position_02, grid_constant=grid_constant, illumination=illumination
)

# Fix nan or inf
precomputed_heatmaps = np.nan_to_num(precomputed_heatmaps)
precomputed_heatmaps_02 = np.nan_to_num(precomputed_heatmaps_02)

# combine single side and dual side
precomputed_heatmaps = precomputed_heatmaps + precomputed_heatmaps_02

In [ ]:
# Function to update the plots based on selected centered y and z positions
def update_plot_xy(min_z_position, z_position, y_position, grid_constant=1.0, contour=False, low_contour_percent=75, high_contour_percent=75):
    global last_fig  # To store the figure globally

    # Create a figure with GridSpec
    fig = plt.figure(figsize=(12, 6))
    gs = GridSpec(1, 3, width_ratios=[6, 6, 0.3]) 

    reference_value = np.max(precomputed_heatmaps[min_z_position])

    # First plot: XY plane at the selected z position with red line at the centered y position
    ax0 = fig.add_subplot(gs[0])
    heatmap = ax0.imshow(precomputed_heatmaps[z_position], cmap='cividis', 
               extent=[
                   -precomputed_heatmaps.shape[2]*grid_constant//2, 
                   precomputed_heatmaps.shape[2]*grid_constant//2, 
                   -precomputed_heatmaps.shape[1]*grid_constant//2, 
                   precomputed_heatmaps.shape[1]*grid_constant//2
               ],
               vmin=0,vmax=reference_value)
    
    ax0.set_title(f'XY plane at z={z_position}')
    ax0.set_xlabel('X (centered)')
    ax0.set_ylabel('Y (centered)')
    ax0.set_aspect('equal')
    # Add colorbar to the right of the heatmap
    cbar_ax = fig.add_subplot(gs[2])  # Add the colorbar axis
    plt.colorbar(heatmap, cax=cbar_ax)

    ax1 = fig.add_subplot(gs[1])
    heatmap = ax1.imshow(precomputed_heatmaps[:,y_position,:], cmap='cividis', 
               extent=[
                   -precomputed_heatmaps.shape[2]*grid_constant//2, 
                   precomputed_heatmaps.shape[2]*grid_constant//2, 
                   0, 
                   precomputed_heatmaps.shape[0]
               ],
               vmin=0,vmax=reference_value)
    ax1.set_title(f'XZ plane at y={y_position}')
    ax1.set_xlabel('X (centered)')
    # ax1.set_ylabel('Z')
    ax1.set_aspect('equal')

    # Overlay the contour at 90% of the max value
    # Calculate the 90% of the max value for the contour level
    if contour:
        
        low_contour_level = low_contour_percent/100. * reference_value
        high_contour_level = high_contour_percent/100. * reference_value
        
        ax0.contour(precomputed_heatmaps[z_position], 
                    levels=[low_contour_level, high_contour_level], colors=['blue','red'], linewidths=1.5, linestyles='dashed', 
                    extent=[
                        -precomputed_heatmaps.shape[2]*grid_constant//2, 
                        precomputed_heatmaps.shape[2]*grid_constant//2, 
                        -precomputed_heatmaps.shape[1]*grid_constant//2, 
                        precomputed_heatmaps.shape[1]*grid_constant//2
                    ])
        
        ax1.contour(precomputed_heatmaps[:,y_position,:], 
                    levels=[low_contour_level, high_contour_level], colors=['blue','red'], linewidths=1.5, linestyles='dashed', 
                    extent=[
                       -precomputed_heatmaps.shape[2]*grid_constant//2, 
                       precomputed_heatmaps.shape[2]*grid_constant//2, 
                       0, 
                       precomputed_heatmaps.shape[0]
                    ])
    
    last_fig = fig

In [ ]:
# Create sliders for z position and centered y position
min_z_slider = widgets.IntSlider(
    min=0, max=precomputed_heatmaps.shape[0] - 1, value=30,
    step=1, description='min Z position'
)

z_slider = widgets.IntSlider(
    min=0, max=precomputed_heatmaps.shape[0] - 1, 
    step=1, description='Z position'
)

y_slider = widgets.IntSlider(
    min=0, max=precomputed_heatmaps.shape[1] - 1, value=100,
    step=1, description='Y position'
)

low_contour_slider = widgets.IntSlider(
    min=0, max=200, value=75, 
    step=1, description='lower contour %'
)

high_contour_slider = widgets.IntSlider(
    min=0, max=200, value=125, 
    step=1, description='upper contour %'
)

# Interactive plot update
interact(update_plot_xy, min_z_position=min_z_slider, z_position=z_slider, y_position=y_slider, grid_constant=fixed(grid_constant), 
         low_contour_percent=low_contour_slider, high_contour_percent=high_contour_slider)

In [ ]:
# Set path to the file
savepath = '/home/tmurakami/Pictures/bleacher/final_design.svg'

# Save to an SVG file
if last_fig:
    last_fig.savefig(savepath, format='svg')
    print("Plot saved as svg")
else:
    print("No figure to export.")